In [3]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
import numpy as np
import re
import lxml
import string
%matplotlib inline

In [8]:
df = open("moviesdircast.txt", 'a')
df.write("title,date,runtime,genre,rating,description,votes,gross\n")

pages = np.arange(1, 7001, 250)
pages = map(str, pages)
years = [str(i) for i in range(2008,2009)]
headers = {"Accept-Language": "en-US, en;q=0.5"}
punctuation = set(string.punctuation)

for year in years:
    sleep(randint(300,480))
    for page in pages:
        response = get('https://www.imdb.com/search/title/?title_type=feature&release_date=' + year +
        '&adult=include&count=250&start=' + page + '&ref_=adv_nxt', headers = headers)
        sleep(randint(8,15))
        if response.status_code != 200:
            print('uh oh, not 200')
        page_html = BeautifulSoup(response.text, 'html.parser')
        
        containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
        for container in containers:
            firstline = container.find("h3", class_="lister-item-header")
            title = firstline.find("a").text.replace(',', '')
            date = re.sub(r"[()]","", firstline.find_all("span")[-1].text).replace(',', '')
            
            try:
                runtime = container.find("span", class_="runtime").text[:-4].replace(',', '')
            except:
                runtime = "NaN"
            try:
                genre = container.find("span", class_="genre").text.rstrip().replace("\n","").split(",")
                genrestr = ""
                for i in range(len(genre)):
                    genrestr += genre[i].strip()
                    if i == (len(genre) - 1):
                        continue
                    genrestr += '~'
                genrestr = genrestr.replace(',', '')
            except:
                genrestr = "NaN"
            try:
                rating = container.find("strong").text.replace(',', '')
            except:
                rating = "NaN"
            desc = container.find_all("p", class_="text-muted")[-1].text.lstrip()
            desc = ''.join([c for c in desc.lower() if not c in punctuation])
            desc = desc.replace('\n','')

            nums = container.find_all("span", attrs={"name": "nv"})
            
            if len(nums) == 2:
                votes = nums[0].text.replace(',', '')
                gross = nums[1].text.replace(',', '')
            elif len(nums) == 1:
                votes = nums[0].text.replace(',', '')
                gross = "NaN"
            else:
                votes = "NaN"
                gross = "NaN"
            movieCast = container.find("p", class_="")
            try:
                casts = movieCast.text.replace("\n","").split('|')
                casts = [x.strip() for x in casts]
                casts = [casts[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
                movieDirector = casts[0]
                movieStars = [x.strip() for x in casts[1].split(",")]
            except:
                #casts = movieCast.text.replace("\n","").strip()
                movieDirector = "NaN"
                movieStars = "NaN"
            movieDirector.replace(',', '~')
            stars = ""
            for i in range(len(movieStars)):
                stars += movieStars[i].replace(",", '')
                if i != len(movieStars) - 1:
                    stars += "~"
            try:
                meta = container.find("span", class_="metascore unfavorable").text.rstrip().replace(",",'')
            except:
                meta = "NaN"
            df.write(title + "," + date + "," + runtime + "," + genrestr + "," + rating + "," + desc + "," + votes + "," + gross + "," + movieDirector + "," + stars + "," + meta + '\n')
df.close()
print("done")       
            
            

done


In [4]:
movieDirector

nan